In [ ]:
from transformers import AutoModel

# 加载模型
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModel.from_pretrained(model_name, cache_dir="/cache/huggingface/hub")

# 计算总参数量
total_params = sum(p.numel() for p in model.parameters())

print(f"模型的总参数量: {total_params}")

# 比较模型参数差异并提取位置

In [4]:
from typing import Union, List
from pprint import pprint
import torch

from transformers import AutoModelForCausalLM


def compare_mlp_params(model1, model2, mlp_layers: Union[str, List[str]]) -> dict:
    """比较多个MLP层的参数差异

    Args:
        mlp_layers: 支持以下格式：
            - 单个层模式: "transformer.h.0.mlp"
            - 多个层模式: ["transformer.h.0.mlp", "transformer.h.5.mlp"]
    """
    # 统一处理为列表格式
    if isinstance(mlp_layers, str):
        target_patterns = [mlp_layers]
    else:
        target_patterns = mlp_layers

    # 多模式参数提取
    def filter_params(model):
        return {
            name: param
            for name, param in model.named_parameters()
            if any(pattern in name for pattern in target_patterns)
        }

    params1 = filter_params(model1)
    params2 = filter_params(model2)

    # 结构一致性检查
    if params1.keys() != params2.keys():
        missing_in_1 = set(params2.keys()) - set(params1.keys())
        missing_in_2 = set(params1.keys()) - set(params2.keys())
        raise ValueError(
            f"模型结构不一致\n"
            f"Model1缺失层: {list(missing_in_1)}\n"
            f"Model2缺失层: {list(missing_in_2)}"
        )

    differences = {}

    for name in params1:
        p1, p2 = params1[name].cpu(), params2[name].cpu()

        if p1.shape != p2.shape:
            raise ValueError(f"形状不匹配: {name} | {p1.shape} vs {p2.shape}")

        if not torch.equal(p1, p2):
            diff_mask = ~torch.isclose(p1, p2, rtol=1e-5, atol=1e-8)
            diff_indices = torch.unique(torch.nonzero(diff_mask)[:, 0])

            differences[name] = {
                "shape": tuple(p1.shape),
                "diff_count": diff_indices.size(0),
                "diff_ratio": diff_indices.size(0) / p1.numel(),
                "diff_indices": diff_indices.tolist(),
            }

    return differences


# 使用示例
if __name__ == "__main__":
    # 单层比较
    # diff = compare_mlp_params(model1_path, model2_path, "transformer.h.0.mlp")
    model1_path = "Qwen/Qwen2.5-0.5B-Instruct"
    model2_path = "/cache/models/loki_reranker_qwen2_5-0-5b-10_invert_real"
    model1 = AutoModelForCausalLM.from_pretrained(
        model1_path, torch_dtype=torch.bfloat16
    )
    model2 = AutoModelForCausalLM.from_pretrained(
        model2_path, torch_dtype=torch.bfloat16
    )
    # for name, param in model1.named_parameters():
    #     print(name)
    target_modules = []
    for idx, layer in enumerate(model1.model.layers):
        module_str = f"model.layers.{idx}.mlp.down_proj.weight"
        target_modules.append(module_str)
    # 多层比较
    diff = compare_mlp_params(model1, model2, mlp_layers=target_modules)
    pprint(diff)

{'model.layers.0.mlp.down_proj.weight': {'diff_count': 13,
                                         'diff_indices': [67,
                                                          70,
                                                          156,
                                                          181,
                                                          208,
                                                          398,
                                                          420,
                                                          490,
                                                          570,
                                                          646,
                                                          781,
                                                          784,
                                                          872],
                                         'diff_ratio': 2.982921170112782e-06,
                                         'sha

# 打印模型结构

In [ ]:
from transformers import AutoModel

# 加载模型
model_name = "/cache/models/loki_reranker_qwen2_5-0-5b-5_real"
model = AutoModel.from_pretrained(model_name)

print(model)

In [ ]:
from safetensors import safe_open
with safe_open("/cache/models/loki_reranker_qwen2_5-0-5b-5/checkpoint-456688/model.safetensors", framework="pt") as f:
    print(f.keys())  # 直接输出所有权重键名

# 还原模型

In [ ]:
from safetensors import safe_open
from transformers import AutoModelForCausalLM
from src.module.loki_linear import LoKILinear
import json
import torch

checkpoint_path = (
    "/cache/models/loki_reranker_qwen2_5-0-5b-5/checkpoint-456688/model.safetensors"
)
target_neurons_path = "target_neurons/Qwen2.5-0.5B-Instruct/5.json"
target_model = "/cache/models/loki_reranker_qwen2_5-0-5b-5/checkpoint-456688"

with open(target_neurons_path, "r", encoding="utf-8") as f:
    data = json.load(f)
trainable_neurons = list(data)
# 重新初始化原始模型结构
original_model = AutoModelForCausalLM.from_pretrained(target_model)


def merge_loki_weights(loki_layer, original_linear):
    # 合并权重矩阵
    merged_weight = torch.zeros_like(original_linear.weight.data)
    merged_weight[loki_layer.active_pos] = loki_layer.active_part.weight.data
    merged_weight[loki_layer.fixed_pos] = loki_layer.fixed_part.weight.data

    # 合并偏置项
    if original_linear.bias is not None:
        merged_bias = torch.zeros_like(original_linear.bias.data)
        merged_bias[loki_layer.active_pos] = loki_layer.active_bias.data
        merged_bias[loki_layer.fixed_pos] = loki_layer.fixed_bias.data

    # 加载参数到原始层
    original_linear.weight.data.copy_(merged_weight)
    if original_linear.bias is not None:
        original_linear.bias.data.copy_(merged_bias)


# 加载检查点文件

# 遍历所有层还原参数
for layer_idx in range(original_model.config.num_hidden_layers):
    # 获取当前层的原始结构
    original_down_proj = original_model.model.layers[layer_idx].mlp.down_proj

    # 加载LoKI层参数
    with safe_open(checkpoint_path, framework="pt") as f:
        # 创建临时LoKI层用于加载参数
        loki_layer = LoKILinear(
            original_down_proj, target_neurons=trainable_neurons[layer_idx]
        )
        loki_layer.load_state_dict(
            {
                "active_part.weight": f.get_tensor(
                    f"model.layers.{layer_idx}.mlp.down_proj.active_part.weight"
                ),
                "fixed_part.weight": f.get_tensor(
                    f"model.layers.{layer_idx}.mlp.down_proj.fixed_part.weight"
                ),
                # "active_bias": f.get_tensor(
                #     f"model.layers.{layer_idx}.mlp.down_proj.active_bias"
                # ),
                # "fixed_bias": f.get_tensor(
                #     f"model.layers.{layer_idx}.mlp.down_proj.fixed_bias"
                # ),
            },
            strict=True,
        )
        weight = f.get_tensor(f"model.layers.{layer_idx}.mlp.down_proj.active_part.weight")

    # 合并参数到原始层
    merge_loki_weights(loki_layer, original_down_proj)

# 保存还原后的模型
original_model.save_pretrained("/cache/models/loki_reranker_qwen2_5-0-5b-5_real")

In [ ]:
from safetensors import safe_open
from transformers import AutoModelForCausalLM
from src.loki.loki_linear import LoKILinear
import json
import torch

checkpoint_path = (
    "/cache/models/loki_reranker_qwen2_5-0-5b-40/model.safetensors"
)
target_neurons_path = "target_neurons/Qwen2.5-0.5B-Instruct/40.json"
target_model = "/cache/models/loki_reranker_qwen2_5-0-5b-40"

with open(target_neurons_path, "r", encoding="utf-8") as f:
    data = json.load(f)
trainable_neurons = list(data)
# 重新初始化原始模型结构
original_model = AutoModelForCausalLM.from_pretrained(target_model)


def merge_loki_weights(loki_layer, original_linear):
    # 合并权重矩阵
    merged_weight = torch.zeros_like(original_linear.weight.data)
    merged_weight[loki_layer.active_pos] = loki_layer.active_weight.data
    merged_weight[loki_layer.fixed_pos] = loki_layer.fixed_weight.data

    # 合并偏置项
    if original_linear.bias is not None:
        merged_bias = torch.zeros_like(original_linear.bias.data)
        merged_bias[loki_layer.active_pos] = loki_layer.active_bias.data
        merged_bias[loki_layer.fixed_pos] = loki_layer.fixed_bias.data

    # 加载参数到原始层
    original_linear.weight.data.copy_(merged_weight)
    if original_linear.bias is not None:
        original_linear.bias.data.copy_(merged_bias)


# 加载检查点文件

# 遍历所有层还原参数
for layer_idx in range(original_model.config.num_hidden_layers):
    # 获取当前层的原始结构
    original_down_proj = original_model.model.layers[layer_idx].mlp.down_proj

    # 加载LoKI层参数
    with safe_open(checkpoint_path, framework="pt") as f:
        # 创建临时LoKI层用于加载参数
        loki_layer = LoKILinear(
            original_down_proj, target_neurons=trainable_neurons[layer_idx]
        )
        loki_layer.load_state_dict(
            {
                "active_weight": f.get_tensor(
                    f"model.layers.{layer_idx}.mlp.down_proj.active_weight"
                ),
                "fixed_weight": f.get_tensor(
                    f"model.layers.{layer_idx}.mlp.down_proj.fixed_weight"
                ),
                "index_map" : loki_layer.index_map
                # "active_bias": f.get_tensor(
                #     f"model.layers.{layer_idx}.mlp.down_proj.active_bias"
                # ),
                # "fixed_bias": f.get_tensor(
                #     f"model.layers.{layer_idx}.mlp.down_proj.fixed_bias"
                # ),
            },
            strict=True,
        )
        weight = f.get_tensor(f"model.layers.{layer_idx}.mlp.down_proj.active_weight")

    # 合并参数到原始层
    merge_loki_weights(loki_layer, original_down_proj)

# 保存还原后的模型
original_model.save_pretrained("/cache/models/loki_reranker_qwen2_5-0-5b-40_real")

# 抑制指定神经元输出

In [1]:
from src.loki.tools import set_zero_weights

set_zero_weights(
    target_neurons_path="/workspace/ftg/target_neurons/Qwen2.5-0.5B-Instruct/20steps/GLOBAL_1_20steps_L.json",
    output_path="/cache/models/suppressed_1_L",
)


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


[2025-04-30 15:59:41,296] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::chrono::_V2::steady_clock::now()@GLIBCXX_3.4.19'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::string::_M_replace_aux(unsigned long, unsigned long, unsigned

In [ ]:
from src.train.qwen_loki import LoKIQwen2ForCausalLM
import json
from transformers import AutoModelForCausalLM
import torch

target_neurons_path = "target_neurons/Qwen2.5-0.5B-Instruct/5.json"
original_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct", torch_dtype=torch.bfloat16
)
with open(target_neurons_path, "r", encoding="utf-8") as f:
    data = json.load(f)
# custom_config = LoKIQwen2Config("Qwen/Qwen2.5-0.5B-Instruct", target_neurons=data)
model = LoKIQwen2ForCausalLM.from_pretrained(
    pretrained_model_name_or_path="Qwen/Qwen2.5-0.5B-Instruct",
    target_neurons=data,
    torch_dtype=torch.bfloat16,
)
print(model)
# 验证第一层权重是否迁移
loki_layer = model.model.layers[0].mlp.down_proj

pretrained_weight = original_model.model.layers[0].mlp.down_proj.weight

combined_weight = torch.zeros_like(pretrained_weight)
combined_weight[loki_layer.active_pos] = loki_layer.active_part.weight.data
combined_weight[loki_layer.fixed_pos] = loki_layer.fixed_part.weight.data
print(torch.allclose(combined_weight, pretrained_weight, atol=1e-6))  # 应输出 True

In [ ]:
from src.loki.qwen_loki import LoKIQwen2ForCausalLM
import json
import torch
from transformers import AutoConfig, AutoModel, Qwen2Config

target_neurons_path = "target_neurons/Qwen2.5-0.5B-Instruct/5.json"

with open(target_neurons_path, "r", encoding="utf-8") as f:
    data = json.load(f)
# custom_config = LoKIQwen2Config("Qwen/Qwen2.5-0.5B-Instruct", target_neurons=data)
model = LoKIQwen2ForCausalLM.from_pretrained(
    pretrained_model_name_or_path="Qwen/Qwen2.5-0.5B-Instruct",
    target_neurons=data,
    torch_dtype=torch.bfloat16,
)
LoKIQwen2ForCausalLM.register_for_auto_class("AutoModelForCausalLM")

# AutoModel.register(Qwen2Config, LoKIQwen2ForCausalLM)
# LoKIQwen2ForCausalLM.register_for_auto_class("AutoModel")
model.save_pretrained("/cache/models/custom-model")

# 自定义模型加载测试

In [ ]:
from transformers import AutoModelForCausalLM
import json
import torch

target_neurons_path = "target_neurons/Qwen2.5-0.5B-Instruct/10.json"

with open(target_neurons_path, "r", encoding="utf-8") as f:
    data = json.load(f)
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="/cache/models/Qwen2.5-3B-Instruct",
    target_neurons=data,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
print(model)

# 自定义模型加载

In [ ]:
from transformers import AutoModelForCausalLM
import torch
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="/cache/models/custom-model-test",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
print(model)

# 加载后Model还原

In [ ]:
from src.loki.loki_linear import restore_original_linears
from transformers import AutoModelForCausalLM
import torch
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="/cache/models/custom-model-test",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
print(model)
model = restore_original_linears(model)
print(model)

# LoKI模型转换为一般模型

In [4]:
from src.loki.tools import restore_loki_model, restore_loki_model_i,restore_loki_model_
# 需要LoKI模型的safetensors
restore_loki_model(
    model_path="/cache/models/LoKI_models/loki_5_a_0506",
    target_neurons_path="/workspace/ftg/target_neurons/Qwen2.5-0.5B-Instruct/7steps/averaged/5.json",
    output_path="/cache/models/LoKI_models/loki_5_a_0506_real",
)
# restore_loki_model_(
#     model_path="/root/train_outputs/llama/ToolACE_spindle_10",
#     target_neurons_path="/workspace/ftg/target_neurons/llama_3_1_8B/10.json",
#     output_path="/cache/models/ToolACE_10",
#     original_model_name="/root/Llama-3.1"
# )
# restore_loki_model_i(
#     model_path="/root/train_outputs/Qwen2.5-0.5B-Instruct/reranker_continuous_filt_max7_train/loki_10_new",
#     target_neurons_path="/workspace/ftg/target_neurons/qwen0.5_i/10.json",
#     output_path="/cache/models/loki_10_new_real",
# )

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Model restored and saved to /cache/models/LoKI_models/loki_5_a_0506_real


# 创建LoKI模型

In [3]:
from src.loki.tools import create_and_save_loki_model, create_and_save_loki_model_i, create_and_save_loki_model_llama

# create_and_save_loki_model(
#     target_neurons_path="/workspace/ftg/target_neurons/Qwen2.5-0.5B-Instruct/7steps/averaged/20.json",
#     save_dir="/cache/models/loki_20_a",
# )
# create_and_save_loki_model_i(
#     target_neurons_path="/workspace/ftg/target_neurons/llama_3_1_8B/10.json",
#     save_dir="/cache/models/loki_8B_10",
#     model_name="/root/Llama-3.1",
# )
create_and_save_loki_model_llama(
    target_neurons_path="/workspace/ftg/target_neurons/llama2/10.json",
    save_dir="/cache/models/loki_llama2_10_a",
    model_name="/cache/models/official_models/Llama-2-7b-hf",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

成功替换层0
成功替换层1
成功替换层2
成功替换层3
成功替换层4
成功替换层5
成功替换层6
成功替换层7
成功替换层8
成功替换层9
成功替换层10
成功替换层11
成功替换层12
成功替换层13
成功替换层14
成功替换层15
成功替换层16
成功替换层17
成功替换层18
成功替换层19
成功替换层20
成功替换层21
成功替换层22
成功替换层23
成功替换层24
成功替换层25
成功替换层26
成功替换层27
成功替换层28
成功替换层29
成功替换层30
成功替换层31
Parameter: model.layers.0.mlp.down_proj.active_weight, Shape: torch.Size([409, 11008])
Parameter: model.layers.1.mlp.down_proj.active_weight, Shape: torch.Size([409, 11008])
Parameter: model.layers.2.mlp.down_proj.active_weight, Shape: torch.Size([409, 11008])
Parameter: model.layers.3.mlp.down_proj.active_weight, Shape: torch.Size([409, 11008])
Parameter: model.layers.4.mlp.down_proj.active_weight, Shape: torch.Size([409, 11008])
Parameter: model.layers.5.mlp.down_proj.active_weight, Shape: torch.Size([409, 11008])
Parameter: model.layers.6.mlp.down_proj.active_weight, Shape: torch.Size([409, 11008])
Parameter: model.layers.7.mlp.down_proj.active_weight, Shape: torch.Size([409, 11008])
Parameter: model.layers.8.mlp.down_proj.active_weight, Sha

/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::chrono::_V2::steady_clock::now()@GLIBCXX_3.4.19'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::string::_M_replace_aux(unsigned long, unsigned long, unsigned

Done.


In [4]:
from datasets import load_dataset
import json

def transform_conversation(example):
    """转换单个对话样本，保留system列"""
    transformed = []
    for turn in example['conversations']:
        new_turn = turn.copy()
        if new_turn["from"] == "assistant":
            new_turn["from"] = "gpt"
        elif new_turn["from"] == "user":
            new_turn["from"] = "human"
        elif new_turn["from"] == "tool":
            new_turn["from"] = "observation"
        transformed.append(new_turn)
    
    # 保留所有原始字段
    return {
        **example,  # 保留原始所有字段
        "conversations": transformed  # 更新转换后的对话
    }

def process_and_save_dataset(dataset_name, save_path):
    """
    处理并保存数据集（保留system列）
    :param dataset_name: 数据集名称或路径
    :param save_path: 保存路径
    """
    # 加载原始数据集
    dataset = load_dataset(dataset_name)
    print(dataset)
    # 应用转换函数
    transformed_dataset = dataset.map(
        function=transform_conversation,
        batched=False
    )
    
    # 保存处理后的数据集
    transformed_dataset.save_to_disk(save_path)
    print(f"数据集已保存至：{save_path}")
    
    return transformed_dataset

# 使用示例
if __name__ == "__main__":
    # 配置参数
    dataset_name = "/workspace/ToolACE"
    save_path = "./transformed_dataset_with_system"
    
    # 处理并保存数据集
    processed_dataset = process_and_save_dataset(dataset_name, save_path)
    
    # 验证样本
    print("\n转换后的样本示例：")
    sample = processed_dataset['train'][0]
    print(f"System: {sample['system']}")
    print("对话片段：")
    print(json.dumps(sample['conversations'][:2], indent=2))
    print(processed_dataset)

DatasetDict({
    train: Dataset({
        features: ['system', 'conversations'],
        num_rows: 11300
    })
})


Saving the dataset (1/1 shards): 100%|██████████| 11300/11300 [00:00<00:00, 205104.88 examples/s]

数据集已保存至：./transformed_dataset_with_system

转换后的样本示例：
System: You are an expert in composing functions. You are given a question and a set of possible functions. 
Based on the question, you will need to make one or more function/tool calls to achieve the purpose. 
If none of the function can be used, point it out. If the given question lacks the parameters required by the function,
also point it out. You should only return the function call in tools call sections.
Here is a list of functions in JSON format that you can invoke:
[{"name": "newAddress", "description": "Generates a new Ethereum address that can be used to send or receive funds. Do not lose the password! We can't restore access to an address if you lose it.", "parameters": {"type": "dict", "properties": {"password": {"description": "The password for the new Ethereum address", "type": "string"}}, "required": ["password"]}, "required": null}, {"name": "Market Trends API", "description": "Get the latest market trends and releva

In [5]:
from datasets import load_dataset
import json
import os

def save_as_json(dataset, save_dir, splits=None):
    """
    将数据集保存为JSON格式
    :param dataset: 处理后的数据集对象
    :param save_dir: 保存目录路径
    :param splits: 需要保存的数据集划分（默认为全部）
    """
    os.makedirs(save_dir, exist_ok=True)
    
    splits = splits or dataset.keys()
    for split in splits:
        file_path = os.path.join(save_dir, f"{split}.json")
        
        # 保存为多行JSON（每行一个样本）
        dataset[split].to_json(
            path_or_buf=file_path,
            orient="records",
            lines=True,        # 每行一个JSON对象
            force_ascii=False  # 支持非ASCII字符
        )
        
        print(f"{split.upper()}数据集已保存至：{file_path}")

# 使用示例（接之前的处理流程）
if __name__ == "__main__":
    # 加载并转换数据集
    dataset_name = "/workspace/ToolACE"
    save_path = "./transformed_dataset"
    processed_dataset = process_and_save_dataset(dataset_name, save_path)
    
    # 另存为JSON格式
    json_save_dir = "./json_dataset"
    save_as_json(processed_dataset, json_save_dir)
    
    # 验证输出
    with open(os.path.join(json_save_dir, "train.json"), 'r') as f:
        sample_line = f.readline()
        print("\nJSON样本示例：")
        print(json.dumps(json.loads(sample_line), indent=2, ensure_ascii=False))

DatasetDict({
    train: Dataset({
        features: ['system', 'conversations'],
        num_rows: 11300
    })
})


Saving the dataset (1/1 shards): 100%|██████████| 11300/11300 [00:00<00:00, 205109.32 examples/s]


数据集已保存至：./transformed_dataset


Creating json from Arrow format: 100%|██████████| 12/12 [00:00<00:00, 21.99ba/s]

TRAIN数据集已保存至：./json_dataset/train.json

JSON样本示例：
{
  "system": "You are an expert in composing functions. You are given a question and a set of possible functions. \nBased on the question, you will need to make one or more function/tool calls to achieve the purpose. \nIf none of the function can be used, point it out. If the given question lacks the parameters required by the function,\nalso point it out. You should only return the function call in tools call sections.\nHere is a list of functions in JSON format that you can invoke:\n[{\"name\": \"newAddress\", \"description\": \"Generates a new Ethereum address that can be used to send or receive funds. Do not lose the password! We can't restore access to an address if you lose it.\", \"parameters\": {\"type\": \"dict\", \"properties\": {\"password\": {\"description\": \"The password for the new Ethereum address\", \"type\": \"string\"}}, \"required\": [\"password\"]}, \"required\": null}, {\"name\": \"Market Trends API\", \"descript